In [3]:
pip install folium

  Using cached branca-0.8.1-py3-none-any.whl.metadata (1.5 kB)
  Using cached xyzservices-2025.1.0-py3-none-any.whl.metadata (4.3 kB)
Using cached branca-0.8.1-py3-none-any.whl (26 kB)
Using cached xyzservices-2025.1.0-py3-none-any.whl (88 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pandas

   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
    --------------------------------------- 0.3/11.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.6 MB 1.3 MB/s eta 0:00:09
   -- ------------------------------------- 0.8/11.6 MB 1.2 MB/s eta 0:00:09
   --- ------------------------------------ 1.0/11.6 MB 1.3 MB/s eta 0:00:09
   ---- ----------------------------------- 1.3/11.6 MB 1.3 MB/s eta 0:00:09
   ----- ---------------------------------- 1.6/11.6 MB 1.3 MB/s eta 0:00:08
   ------ --------------------------------- 1.8/11.6 MB 1.3 MB/s eta 0:00:08
   ------- -------------------------------- 2.1/11.6 MB 1.2 MB/s eta 0:00:08
   -------- ------------------------------- 2.4/11.6 MB 1.3 MB/s eta 0:00:08
   --------- ------------------------------ 2.6/11.6 MB 1.3 MB/s eta 0:00:08
   ---------- -------------

In [16]:
import folium as flm
from folium import plugins
import pandas as pd
import json

# Load Indian States GeoJSON
with open('india.geojson', 'r') as f:
    geojson_states = json.load(f)

# Load COVID-19 Data
df_covid = pd.read_csv('Latest Covid-19 India Status.csv')

# Standardize State Names (Fix Discrepancies)
df_covid.loc[df_covid['State/UTs'] == 'Dadra and Nagar Haveli and Daman and Diu', 'State/UTs'] = 'Dadra and Nagar Haveli'

# Map Data to GeoJSON Features
for feature in geojson_states['features']:
    state_name = feature['properties']['NAME_1']  # State name from GeoJSON
    covid_data = df_covid[df_covid['State/UTs'] == state_name]  # Match state names

    if not covid_data.empty:
        feature['properties']['total_case'] = int(covid_data['Total Cases'].values[0])
        feature['properties']['active_case'] = int(covid_data['Active'].values[0])
        feature['properties']['total_deaths'] = int(covid_data['Deaths'].values[0])
    else:
        feature['properties']['total_case'] = 0
        feature['properties']['active_case'] = 0
        feature['properties']['total_deaths'] = 0

# Initialize Map with Mapbox Tile
mapbox_access_token = "pk.eyJ1IjoicmFnaGF2MjIiLCJhIjoiY203dTBlMmxjMTJyNTJqc2VubnI1M3M3MCJ9.Rpkhfufz90308aqqNoJ6Fw"
map1 = flm.Map(
    location=[20.5937, 78.9629],
    zoom_start=5,
    tiles=f"https://api.mapbox.com/styles/v1/mapbox/light-v10/tiles/256/{{z}}/{{x}}/{{y}}?access_token={mapbox_access_token}",
    attr="Mapbox"
)

# Add Choropleth Layer for COVID-19 Cases
choropleth = flm.Choropleth(
    geo_data=geojson_states,
    name="COVID-19 Cases",
    data=df_covid,
    columns=["State/UTs", "Total Cases"],
    key_on="feature.properties.NAME_1",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name="Total COVID-19 Cases"
).add_to(map1)

# Add Click Feature to Show Data
def style_function(feature):
    return {
        "fillColor": "#ff7800",
        "color": "black",
        "weight": 2,
        "fillOpacity": 0.5
    }

geojson_layer = flm.GeoJson(
    geojson_states,
    style_function=style_function,
    tooltip=flm.GeoJsonTooltip(
        fields=["NAME_1", "total_case", "active_case", "total_deaths"],
        aliases=["State", "Total Cases", "Active Cases", "Total Deaths"],
        localize=True
    )
).add_to(map1)

# Add Layer Control
flm.LayerControl().add_to(map1)

# Display Map
map1


KeyError: 'NAME_1'

In [18]:
import json

# Load GeoJSON file
with open("india.geojson", "r", encoding="utf-8") as f:
    geojson_states = json.load(f)

# Print the structure of one feature
print(json.dumps(geojson_states["features"][0], indent=4))


{
    "type": "Feature",
    "properties": {
        "district": "Aizawl",
        "dt_code": "261",
        "st_nm": "Mizoram",
        "st_code": "15",
        "year": "2011_c"
    },
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [
                    93.04466353722471,
                    23.41052458365152
                ],
                [
                    92.9467955185,
                    23.51363116309284
                ],
                [
                    92.88104044341934,
                    23.51913018066304
                ],
                [
                    92.78164323690206,
                    23.299169477854896
                ],
                [
                    92.68989196934763,
                    23.328039320098465
                ],
                [
                    92.67307090362932,
                    23.383029495800503
                ],
                [
                    92.654720

In [20]:
state_name = feature['properties'].get("state_name", "Unknown")  # NEW (Flexible)
